In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

## Step A - Multiple Product Detection

In [ ]:
query_images = ['models/0.jpg','models/1.jpg','models/11.jpg','models/19.jpg','models/24.jpg','models/25.jpg','models/26.jpg']
scene_images = ['scenes/e1.png','scenes/e2.png','scenes/e3.png','scenes/e4.png','scenes/e5.png']
true_images = {
    'scenes/e1.png':{'models/0.jpg','models/11.jpg'},
    'scenes/e2.png':{'models/24.jpg','models/25.jpg','models/26.jpg'},
    'scenes/e3.png':{'models/0.jpg','models/1.jpg','models/11.jpg'},
    'scenes/e4.png':{'models/0.jpg','models/11.jpg','models/25.jpg','models/26.jpg'},
    'scenes/e5.png':{'models/19.jpg','models/25.jpg'},
}


In [ ]:
MM = []
NM = []
for scene in scene_images:

    img2_bgr = cv2.imread(scene)
    img2 = cv2.cvtColor(img2_bgr,cv2.COLOR_BGR2RGB)

    for img in query_images:
        img1_bgr = cv2.imread(img)
        img1 = cv2.cvtColor(img1_bgr,cv2.COLOR_BGR2RGB)

        sift = cv2.xfeatures2d.SIFT_create()

        kp1, des1 = sift.detectAndCompute(img1,None)
        kp2, des2 = sift.detectAndCompute(img2,None)

        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
        search_params = dict(checks = 50)

        flann = cv2.FlannBasedMatcher(index_params, search_params)

        matches = flann.knnMatch(des1,des2,k=2)

        good = []
        for m,n in matches:
            if m.distance < 0.7*n.distance:
                good.append(m)
        if img in true_images[scene]:
            MM.append(len(good))
        else:
            NM.append(len(good))

In [ ]:
accuracy = []
precision = []
my_range = range(0,max(MM))
for par in my_range:
    TP = sum(1 for x in MM if x>=par)
    FN = sum(1 for x in MM if x<par)
    TN = sum(1 for x in NM if x<par)
    FP = sum(1 for x in NM if x>=par)  
    accuracy.append((TP+TN)/(TP+FP+TN+FN)*100)
    precision.append(TP/(TP+FP)*100)

In [ ]:
fig, ax1 = plt.subplots()

fig.set_size_inches(10,5)

color = 'tab:red'
ax1.set_xlabel('MIN_MATCH_COUNT')
ax1.set_ylabel('accuracy', color=color)
ax1.plot(my_range, accuracy, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()

color = 'tab:blue'
ax2.set_ylabel('precision', color=color)
ax2.plot(my_range, precision, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()
plt.show();

print("Optimal MIN_MATCH_COUNT: " + str(accuracy.index(max(accuracy))))

In [ ]:
MIN_MATCH_COUNT = accuracy.index(max(accuracy))

problems = set()
TTP = 0
TFP = 0
TTN = 0
TFN = 0

for scene in scene_images:

    TP = 0
    FP = 0
    TN = 0
    FN = 0
    
    img2_bgr = cv2.imread(scene)
    img2 = cv2.cvtColor(img2_bgr,cv2.COLOR_BGR2RGB)
    
    print("Scene: " + scene + "\n")
    
#    plt.imshow(img2),plt.show();
    
    for img in query_images:
        img1_bgr = cv2.imread(img)
        img1 = cv2.cvtColor(img1_bgr,cv2.COLOR_BGR2RGB)
#        img2_bgr = cv2.imread(scene)
#        img2 = cv2.cvtColor(img2_bgr,cv2.COLOR_BGR2RGB)

        sift = cv2.xfeatures2d.SIFT_create()

        kp1, des1 = sift.detectAndCompute(img1,None)
        kp2, des2 = sift.detectAndCompute(img2,None)

        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
        search_params = dict(checks = 50)

        flann = cv2.FlannBasedMatcher(index_params, search_params)

        matches = flann.knnMatch(des1,des2,k=2)

        good = []
        for m,n in matches:
            if m.distance < 0.7*n.distance:
                good.append(m)

        if len(good)>MIN_MATCH_COUNT:
            src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
            dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
            M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
            matchesMask = mask.ravel().tolist()
            h,w,d = img1.shape
            pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
            dst = cv2.perspectiveTransform(pts,M)
            draw_params = dict(matchColor = (0,0,0), # draw matches in green color
                           singlePointColor = None,
                           matchesMask = matchesMask, # draw only inliers
                           flags = 2)
#            print( "Product " + img + ": enough matches are found!!! - {}/{}".format(len(good), MIN_MATCH_COUNT) )
#            img3 = cv2.drawMatches(img1,kp1,img2,kp2,good,None,**draw_params)
#            plt.figure(figsize = (15,15))
#            plt.imshow(img3),plt.show();
            if img in true_images[scene]:
                TP += 1
                TTP += 1
                img2 = cv2.polylines(img2,[np.int32(dst)],True,(0,255,0),3, cv2.LINE_AA)
                print("TP " + img + " (" + str(len(good)) + " matches)")
            else:
                FP += 1
                TFP += 1
                img2 = cv2.polylines(img2,[np.int32(dst)],True,(255,0,0),3, cv2.LINE_AA)
                print("\033[1m" + "FP " + img + " (" + str(len(good)) + " matches)\033[0m")
                problems.add(img)
        else:
#            print( "Product " + img + ": not enough matches are found - {}/{}".format(len(good), MIN_MATCH_COUNT) )
            matchesMask = None
            if img in true_images[scene]:
                FN += 1
                TFN += 1
                print("\033[1m" + "FN " + img + " (" + str(len(good)) + " matches)\033[0m")
                problems.add(img)
            else:
                TN += 1
                TTN += 1
                print("TN " + img + " (" + str(len(good)) + " matches)")
    print("\n" + "TP: " + str(TP) + "\t" + "FP: " + str(FP) + "\t" + "TN: " + str(TN) + "\t" + "FN: " + str(FN) + "\n")
    print("Accuracy: %.2f" % ((TP+TN)/(TP+FP+TN+FN)*100) + "%\t"+"Precision: %.2f" % (TP/(TP+FP)*100) + "%")
    print("Sensitivity: %.2f" % (TP/(TP+FN)*100) + "%\t"+"Specificity: %.2f" % (TN/(TN+FP)*100) + "%")
    print("F score: %.2f" % (2*((TP/(TP+FP))*(TP/(TP+FN)))/((TP/(TP+FP))+(TP/(TP+FN)))))
    plt.imshow(img2),plt.show();
    print("\n")
print("\n\033[1m" + "TTP: " + str(TTP) + "\t" + "TFP: " + str(TFP) + "\t" + "TTN: " + str(TTN) + "\t" + "TFN: " + str(TFN) + "\033[0m\n")
print("\033[1m" + "Total accuracy: %.2f" % ((TTP+TTN)/(TTP+TFP+TTN+TFN)*100) + "%\t\t"+"Total precision: %.2f" % (TTP/(TTP+TFP)*100) + "%\033[0m")
print("\033[1m" + "Total sensitivity: %.2f" % (TTP/(TTP+TFN)*100) + "%\t"+"Total specificity: %.2f" % (TTN/(TTN+TFP)*100) + "%\033[0m")
print("\033[1m" + "Total F score: %.2f" % (2*((TTP/(TTP+TFP))*(TTP/(TTP+TFN)))/((TTP/(TTP+TFP))+(TTP/(TTP+TFN)))) + "\033[0m")

print("\n\n" + "Problematic images: ")
for img in problems:
    img1_bgr = cv2.imread(img)
    img1 = cv2.cvtColor(img1_bgr,cv2.COLOR_BGR2RGB)
    plt.imshow(img1),plt.show();

## STEP B 

In [ ]:
def buildRefTable(model): # OFFLINE PHASE
   
    kp_model, des_model = sift.detectAndCompute(model,None)


    center_x = np.mean(list(kp_model[i].pt[0] for i in range(len(kp_model))))
    center_y = np.mean(list(kp_model[i].pt[1] for i in range(len(kp_model))))


    table={}
    for kp, des in zip(kp_model, des_model):
        v = ((center_x-kp.pt[0]), (center_y-kp.pt[1]))
        table[tuple(des)]=tuple((kp.pt,kp.angle,kp.size,v))

    return table,des_model

In [ ]:
sift = cv2.xfeatures2d.SIFT_create()

model_bgr = cv2.imread('models/0.jpg')
#model = cv2.cvtColor(model_bgr,cv2.COLOR_BGR2RGB)
model = cv2.cvtColor(model_bgr,cv2.COLOR_BGR2GRAY)

ref_table,des_model=buildRefTable(model)
kp_model, des_model = sift.detectAndCompute(model,None)

In [ ]:
#train_bgr = cv2.imread('scenes/e1.png')
#train = cv2.cvtColor(train_bgr,cv2.COLOR_BGR2RGB)
train_gray = cv2.imread('scenes/h5.jpg',0)
train = cv2.cvtColor(train_gray,cv2.COLOR_GRAY2RGB)



kp_train, des_train = sift.detectAndCompute(train,None)

FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)

flann = cv2.FlannBasedMatcher(index_params, search_params)

matches = flann.knnMatch(des_model,des_train,k=2)

good_matches = []
for m,n in matches:
    if m.distance < n.distance:
        good_matches.append(m)



In [ ]:
matches_4d = []
for m in good_matches:
    entry_model = ref_table[tuple(des_model[m.queryIdx])]
    entry_train = kp_train[m.trainIdx]

    angle = entry_train.angle - entry_model[1]
    scale = entry_train.size / entry_model[2]
    x_c = entry_train.pt[0] + scale*(np.cos(angle)*entry_model[3][0] - np.sin(angle)*entry_model[3][1])
    y_c = entry_train.pt[1] + scale*(np.sin(angle)*entry_model[3][0] + np.cos(angle)*entry_model[3][1])

    matches_4d.append((x_c,y_c,angle,scale))

In [ ]:
X_BINS=10
Y_BINS=8
ANGLE_BINS=3
SCALE_BINS=3

x_max,x_min = train.shape[1]*1.1, -train.shape[1]*0.1
y_max,y_min = train.shape[0]*1.1, -train.shape[0]*0.1
angle_max,angle_min = 30, -30
scale_max,scale_min = 4, 0

def votesOnMatch(m_4d,train):

    accumulator = np.zeros((X_BINS,Y_BINS,ANGLE_BINS,SCALE_BINS))

    for m in m_4d:
        try:
            accumulator[int((m[0]-x_min)*X_BINS//(x_max-x_min))][int((m[1]-y_min)*Y_BINS//(y_max-y_min))][int((m[2]-angle_min)*ANGLE_BINS//(angle_max-angle_min))][int((m[3]-scale_min)*SCALE_BINS//(scale_max-scale_min))]+=1
#            accumulator[int((m[0]-x_min)*X_BINS//(x_max-x_min))+1][int((m[1]-y_min)*Y_BINS//(y_max-y_min))+1][int((m[2]-angle_min)*ANGLE_BINS//(angle_max-angle_min))+1][int((m[3]-scale_min)*SCALE_BINS//(scale_max-scale_min))+1]+=1
        except: pass
        
    return accumulator  

In [ ]:
accumulator=votesOnMatch(matches_4d,train)

In [ ]:
print(sorted(np.ravel(accumulator)))

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(111, projection='3d')
x=[]
y=[]
z=[]
c=[]
for m in matches_4d:
    x.append(m[0])
    y.append(m[1])
    z.append(m[2])
    c.append(m[3])
    

img = ax.scatter(x, y, z, c=c, cmap=plt.hot())
fig.colorbar(img)
plt.show()

In [ ]:
THRESHOLD = 20
found_matches = np.argwhere(accumulator>THRESHOLD)
better_matches = []
for j in range(len(found_matches)):
    f = found_matches[j]
    better_matches.append([])
    for i in range(len(matches_4d)):
        m = matches_4d[i]
        if m[0] >= x_min+f[0]*(x_max-x_min)/X_BINS and m[0] < x_min+(f[0]+1)*(x_max-x_min)/X_BINS\
        and m[1] >= y_min+f[1]*(y_max-y_min)/Y_BINS and m[1] < y_min+(f[1]+1)*(y_max-y_min)/Y_BINS\
        and m[2] >= angle_min+f[2]*(angle_max-angle_min)/ANGLE_BINS and m[2] < angle_min+(f[2]+1)*(angle_max-angle_min)/ANGLE_BINS\
        and m[3] >= scale_min+f[3]*(scale_max-scale_min)/SCALE_BINS and m[3] < scale_min+(f[3]+1)*(scale_max-scale_min)/SCALE_BINS:
            better_matches[j].append(good_matches[i])  

In [ ]:
for matches in better_matches:
    src_pts = np.float32([ kp_model[m.queryIdx].pt for m in matches ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp_train[m.trainIdx].pt for m in matches ]).reshape(-1,1,2)
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
    matchesMask = mask.ravel().tolist()
    h,w = model.shape
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    dst = cv2.perspectiveTransform(pts,M)
    draw_params = dict(matchColor = (0,0,0), # draw matches in green color
                   singlePointColor = None,
                   matchesMask = matchesMask, # draw only inliers
                   flags = 2)
    train = cv2.polylines(train,[np.int32(dst)],True,(0,255,0),3, cv2.LINE_AA)
plt.figure(figsize = (15,15))
plt.imshow(train),plt.show();

In [ ]:
# x_bins=[((x_max-x_min)/X_BINS)*i+x_min for i in range(X_BINS+1)]
# y_bins=[((y_max-y_min)/Y_BINS)*i+y_min for i in range(Y_BINS+1)]
# angle_bins=[((angle_max-angle_min)/ANGLE_BINS)*i+angle_min for i in range(ANGLE_BINS+1)]
# scale_bins=[((scale_max-scale_min)/SCALE_BINS)*i+scale_min for i in range(SCALE_BINS+1)]

In [ ]:
# model part 

sift = cv2.xfeatures2d.SIFT_create()

model_bgr = cv2.imread('models/0.jpg')
model = cv2.cvtColor(model_bgr,cv2.COLOR_BGR2RGB)

kp_model, des_model = sift.detectAndCompute(model,None)


center_x, center_y= np.mean(list(kp_model[i].pt[0] for i in range(len(kp_model)))), np.mean(list(kp_model[i].pt[1] for i in range(len(kp_model))))




# Number of keypoints
print(len(kp_model))

# Location of the keypoint
print(kp_model[0].pt)

# Scale
print(kp_model[0].size)

# Rotation
print(kp_model[0].angle)


In [ ]:

train_bgr = cv2.imread('scenes/e1.png')
train = cv2.cvtColor(train_bgr,cv2.COLOR_BGR2RGB)

kp_train, des_train = sift.detectAndCompute(train,None)

FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)

flann = cv2.FlannBasedMatcher(index_params, search_params)

matches = flann.knnMatch(des1,des2,k=2)
